In [18]:
#data / numbers
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import string


#NLP
import gensim
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import MinMaxScaler


#PCA
from sklearn.decomposition import PCA
import contractions

#viz

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, iplot, plot
import chart_studio
import plotly.figure_factory as ff
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='kitsamho', api_key='JPuAu5Xtf7BhX43kDd4m')

import tensorflow as tf
import tensorflow_hub as hub

import glob




In [ ]:
"""
GITHUB
readme
1) scraper as class
2) Main notebook
3) raw data




MEDIUM
copy
gists
some codeblocks


"""

### Read in data..

In [200]:
df = pd.read_csv('lyrics_final_edit.csv') #read in csv

### Get Genres

In [201]:
df.artist.unique()

array(['Snoop Dogg', 'Cannibal Corpse', 'Kanye West', 'Nine Inch Nails',
       'AC DC', 'The Rolling Stones', 'Public Enemy', 'Alice In Chains',
       'Cradle of Filth', 'Metallica', 'The Beatles', 'Slayer', 'Adele',
       'JAY Z', 'Nirvana', 'Green Day', 'Motley Crue', 'Stevie Wonder',
       'Sex Pistols', 'Elton John', 'Queen', 'Stormzy', 'Megadeth',
       'Madonna', 'U2', 'Drake', 'Coldplay'], dtype=object)

In [202]:
genres = ['hip-hop/rap','death-metal','hip-hop/rap',
          'alternative','rock','rock','hip-hop/rap',
          'alternative','death-metal','metal',
          'pop','metal','soul','hip-hop/rap','alternative','alternative',
          'rock','soul',
          'punk','pop','rock','hip-hop/rap','metal',
          'pop','pop/rock','hip-hop/rap','pop/rock']

In [203]:
#make a genre dictionary
genre_dic = dict(zip(df.artist.unique(),genres))

In [204]:
df['genre'] = df.artist.map(lambda x: genre_dic[x])

### Count word frequencies for each song

In [205]:
df['lyric_count'] = df['lyrics'].map(lambda x: len(x.split()))

In [206]:
norm = [float(i)/max(df.lyric_count) for i in df.lyric_count]

In [207]:
df['lyric_count_norm'] = norm

In [208]:
df['lyric_count_norm'] = df['lyric_count_norm'].map(lambda x: x*55)

### Get a unique artist code (for plotting)

In [209]:
artist_code = [i for i in range(len(df.artist.unique()))]

artist_dic = dict(zip(df.artist.unique(),artist_code))

df['artist_coding'] = df.artist.map(lambda x: artist_dic[x])

### Get a genre code (for plotting)

In [210]:

genre_code = [i for i in range(len(df.genre.unique()))]

genre_code_dic = dict(zip(df.genre.unique(),genre_code))

df['genre_coding'] = df.genre.map(lambda x: genre_code_dic[x])


In [212]:
df.tail(2)

,artist,song,lyrics,genre,lyric_count,lyric_count_norm,artist_coding,genre_coding
1008,Coldplay,When I Need A Friend,"Holy, Holy Dove descend Soft and slowly When I...",pop/rock,72,2.600131,26,8
1009,Coldplay,Life in Technicolor II,There's a wild wind blowing Down the corner of...,pop/rock,117,4.225213,26,8


# EDA

Explore distribution of genres in data

In [253]:
fig = pd.DataFrame(df.groupby('genre')['genre'].count()).plot.bar(template='simple_white',labels=dict(index="genre", value="count"))
fig.show()

Explore song frequency by artist

In [245]:
pd.DataFrame(df.artist.value_counts()).plot.bar(title='test',template='simple_white',
                                                labels=dict(index="band", value="count"))

Let's look at the distribution of word counts in songs by genre

In [254]:
lyric_count_df = []

In [255]:
for i in df.genre.unique():
    
    lyric_count_df.append(pd.DataFrame(df[df.genre == i]['lyric_count']))
    
word_counts = pd.concat([i for i in lyric_count_df],axis=1)

In [256]:
word_counts.columns = df.genre.unique()

In [261]:
word_counts.plot.box(labels=dict(variable="genre", value="count"),template='simple_white',title='Distribution of lyric length by genre')

# Load Universal Sentence Encoder

In [64]:
USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [65]:
def getUSEEmbed(_string,USE = USE):
    
    return np.array(USE([_string])[0])

# USE Embeddings

In [66]:
#get USE embeddings for each song

df_embed = pd.DataFrame([getUSEEmbed(df.lyrics[i]) for i in range(df.shape[0])],\
                        index=[df.song[i] for i in range(df.shape[0])])

In [67]:
def getPCA(df,n_components):
    
    embeddings = df.iloc[:, 1:] #subset on embeddings only
    
    pca = PCA(n_components=n_components) #instantiate PCA

    pca.fit(embeddings) #fit the embeddings

    new_values = pca.transform(embeddings) #transform / reduce data
    
    #option to have 1,2 columns / components
    if n_components == 2:
        
        columns = ['pca_x','pca_y']
        
    elif n_components == 3:
        
        columns = ['pca_x','pca_y','pca_z']
        
    else: 
        columns = ['pc']
    
    df_reduced = pd.DataFrame(new_values,index=df.index,columns=columns) #get new DataFrame
    
    if n_components == 1:
        
        df_reduced.sort_values(by='pc',ascending=False,inplace=True)
    
    return df_reduced #return new DataFrame

In [262]:
df_reduced = getPCA(df_embed,1)

In [263]:
df_merge = pd.merge(df,df_reduced,how='inner',left_on='song',right_on=df_reduced.index)

In [272]:
pd.DataFrame(df_merge.groupby('artist')['pc'].mean().sort_values()).plot.bar(\
                            labels=dict(variable="genre", value="PC-1"),template='simple_white',title='Similarity of lyrics by artist')

# Plotting

In [91]:
def get2Plot(df,title,colour_col,label_col,x='pca_x',y='pca_y',sizing_factor=10,font=10):


        data=go.Scatter(opacity=0.7,name=False,visible=True,
                x=df[x],
                y=df[y],
                mode='markers+text',textposition='middle right',text=df[label_col],
                            marker=dict(size=sizing_factor,color=df[colour_col]))

        # set up axis for plot
        axis=dict(showline=True, # hide axis line, grid, ticklabels and  title
                  zeroline=True,
                  showgrid=False,
                  showticklabels=True,
                  title=''
                  )

        #set up figure for plot
        figure = {
        "data": data,
        "layout":
        go.Layout(title=title,
                    font= dict(family='Open Sans',size=font),
                    width=900,
                    height=900,
                    autosize=False,
                    showlegend=False,
                    xaxis=axis,
                    yaxis=axis,
                    margin=dict(
                    l=40,
                    r=40,
                    b=85,
                    t=100,
                    pad=0,),)}


        #plot the figure
        return iplot(figure)

In [92]:
get2Plot(df_merge,title='test',colour_col='genre_coding',label_col='genre',font=1,sizing_factor=df_merge.lyric_count_norm)

In [81]:
def get3Plot(df,title,colour_col,label_col,x='pca_x',y='pca_y',z='pca_z',sizing_factor=df.lyric_count_norm,font=10):


        data=go.Scatter3d(opacity=0.7,name=False,visible=True,
                x=df[x],
                y=df[y],
                z=df[z],
                mode='markers',textposition='middle right',text=df[label_col],
                            marker=dict(size=sizing_factor,color=df[colour_col]))

        # set up axis for plot
        axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
                  zeroline=False,
                  showgrid=False,
                  showticklabels=True,
                  title=''
                  )

        #set up figure for plot
        figure = {
        "data": data,
        "layout":
        go.Layout(title=title,
                    font= dict(family='Open Sans',size=font),
                    width=900,
                    height=900,
                    autosize=False,
                    showlegend=False,
                    xaxis=axis,
                    yaxis=axis,
                    margin=dict(
                    l=40,
                    r=40,
                    b=85,
                    t=100,
                    pad=0,),)}


        #plot the figure
        return iplot(figure)

In [82]:
get3Plot(df_merge,title='test',colour_col='artist_coding',label_col='genre',font=8,sizing_factor=df_merge.lyric_count_norm)

In [331]:
def savePlot(fig,filename):
        
    """Method to save Plotly figure to chart studio"""

    py.plot(fig, filename = filename, auto_open=True)

    return print('Successfully saved as ',filename)

In [418]:
x = similaritySorter(black_lyrics,nlp)


In [ ]:

# #instantiate the spacy model - we need the large instance as this has the pre-trained vectors
nlp = spacy.load('en_core_web_lg')

In [417]:
x = similaritySorter(kill_lyrics,nlp)
getPlot(x,'test')

In [413]:
def similaritySorter(_string,model,n_components=2):
    
    """This function takes in a string and returns a DataFrame summary of the ouputs from a PCA
    The DataFrame can be plotted - (scatter) to see which words in the string have greater 
    similarity"""
    
    text.ENGLISH_STOP_WORDS
    #removes punctuation and expands contractions
    string_clean = contractions.fix(_string.translate(str.maketrans(' ', ' ', string.punctuation)))
        
    token_raw = model(' '.join(set(string_clean.lower().split()))) #tokenises unique tokens
    
    token_list_no_punc = [token for token in token_raw if token.pos_ != 'PUNCT'] #subset of tokens
    
    token_list = [token for token in token_list_no_punc if token.text not in text.ENGLISH_STOP_WORDS]
    
    #DataFrame of vectors
    df = pd.DataFrame([token.vector for token in token_list],index=[token.text for token in token_list])
    
    embeddings = df.iloc[:, 1:] #subset on embeddings only
    
    n_components = n_components #set components
    
    pca = PCA(n_components=n_components) #instantiate PCA

    pca.fit(embeddings) #fit the embeddings

    new_values = pca.transform(embeddings) #transform / reduce data
    
    #option to have 1,2 columns / components
    if n_components == 2:
        
        columns = ['pca_x','pca_y']
        
    else: 
        
        columns = ['pc']
    
    df_reduced = pd.DataFrame(new_values,index=df.index,columns=columns) #get new DataFrame
    
    if n_components == 1:
        
        df_reduced.sort_values(by='pc',ascending=False,inplace=True)
    
    return df_reduced #return new DataFrame